# MODELACIÓN EN HIDROLOGÍA
## Primer Parcial - Parte práctica (70%)

Su equipo de trabajo ha sido contratado para evaluar los efectos del cambio climático en el año 2050 sobre la disponibilidad de agua subterránea en una zona agrícola de Colombia. A continuación, se desarrolla un ejemplo muy simplificado de flujo de trabajo para cumplir con el objetivo del proyecto.

**Parte 1 - Corrección del sesgo con el método de mapeo de distribuciones (25 puntos)**

Usted cuenta con los siguientes archivos para las series de precipitación y temperatura promedio diarias simuladas con un modelo climático regional para el año 2050 en su área de estudio considerando una trayectoria de emisión de Gases Efecto Invernadero (GEI) particular:

- Precipitación: archivo *precip2050.csv*
- Temperatura: archivo *temp2050.csv*

La precipitación está en unidades de mm/d y la temperatura en K.

Un colega de su equipo de modelación le hace entrega de los parámetros de las distribuciones *gamma* y *normal* ajustadas para los datos simulados y observados de precipitación y temperatura durante el período de control:

**Parámetros distribución *gamma***

| Parámetro | Serie observada | Serie simulada |
| --------- | --------------- | -------------- |
| k         | 0.25            | 0.4            | 
| $\theta$  | 6.0             | 4.0            |

**Parámetros distribución *normal*** (valores en grados Celsius)

| Parámetro | Serie observada | Serie simulada |
| --------- | --------------- | -------------- |
| $\mu$     | 18.3            | 17.8           | 
| $\sigma$  | 2.2             | 2.9            |


**Nota:** En la práctica, es deseable tener los parámetros para cada mes. No obstante, para fines de este examen, se usará una sola distribución para todos los días del año.

Su primera tarea consiste en utilizar el método de mapeo de distribuciones para corregir el sesgo de las series simuladas en el año 2050. En el siguiente bloque usted ya cuenta con las funciones necesarias para llevar a cabo esta tarea. El procedimiento es el siguiente:

1. Importe los archivos con las series simuladas de precipitación y temperatura <br>
2. Para cada serie de datos, aplique el siguiente procedimiento: <br>
    a) Para cada valor simulado, use la función de probabilidad acumulada de los **datos simulados** y encuentre el valor de probabilidad acumulada respectivo. <br>
    b) Utilizando el valor obtenido en el paso a), utilice la función inversa de probabilidad acumulada de los **datos observados**. De esta forma encuentra el valor corregido. <br>

Recuerde, la distribución *gamma* se usa para la serie de **precipitación** y la distribución *normal* para la serie de **temperatura**. 

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gamma, norm
import matplotlib.pyplot as plt

def gamma_CFD(v, k, theta):
    '''
        Parámetros
        ----------
        v : float
            valor de la variable aleatoria
        k : float
            parámetro de forma
        theta : float
            parámetro de escala

        Returns
        -------
        float
        Cuantil (probabilidad acumulada)
    '''
    return gamma.cdf(v, a=k, scale=theta)

def gamma_invCFD(perc, k, theta):
    '''
        Parámetros
        ----------
        perc : float
            cuantil
        k : float
            parámetro de forma
        theta : float
            parámetro de escala

        Returns
        -------
        float
        valor de la variable aleatoria
    '''
    return gamma.ppf(perc, a=k, scale=theta)

def normal_CFD(v, mu, sigma):
    '''
        Parámetros
        ----------
        v : float
            valor de la variable aleatoria
        mu : float
            media
        sigma : float
            desviación estándar

        Returns
        -------
        float
        Cuantil (probabilidad acumulada)
    '''
    return norm.cdf(v, loc=mu, scale=sigma)

def normal_invCFD(perc, mu, sigma):
    '''
        Parámetros
        ----------
        perc : float
            cuantil
        mu : float
            media
        sigma : float
            desviación estándar

        Returns
        -------
        float
        valor de la variable aleatoria
    '''
    return norm.ppf(perc, loc=mu, scale=sigma)

Desarrolle en el siguiente bloque su código:

In [ ]:
# Importar series simuladas de precipitación y temperatura

Psim = pd.read_csv('precip2050.csv')
Tsim = pd.read_csv('temp2050.csv')

# Mapeo de distribución para precipitación

# ---- programe aquí el método de mapeo de distribución ----------------------
values = Psim['value'] # Así se extraen los valores del DataFrame
new = []

# ----------------------------------------------------------------------------

Pcorr = Psim.copy() # Copie la serie original
Pcorr['value'] = new # Actualice los nuevos valores corregidos
    
# Mapeo de distribución para temperatura

# ---- programe aquí el método de mapeo de distribución ----------------------

# ----------------------------------------------------------------------------

**Parte 2 - Estimación de la evapotranspiración potencial mensual (25 puntos)**

El colega de su equipo de modelación que hizo la modelación climática regional, le hizo entrega de las siguientes series mensuales ya corregidas para el año 2050 en su área de estudio:

- Velocidad del viento: archivo *vel_viento2050.csv*
- Radiación neta: archivo *radiacion2050.csv*
- Humedad relativa: archivo *humedad2050.csv*

La velocidad del viento está en unidades de m/s, la radiación neta en MJ/m$^2$/d y la humedad relativa en %. Suponga que la velocidad del viento y la humedad relativa corresponden a una elevación de 2 m sobre la superficie. La presión atmosférica promedio a nivel de superficie en su área de estudio es 570 mb. 

Con esa información, el lider del equipo de modelación le pide estimar la evapotranspiración de referencia utilizando el método de Penman-Monteith de la FAO. Un colega le hace entrega además de los coeficientes de cultivo efectivos para cada mes con el fin de que estime la evapotranspiración potencial en el área de estudio. Dichos coeficientes están asignados a la variable tipo lista **kc**.

En el siguiente bloque se encuentran las funciones requeridas para hacer el cálculo de la $ET_0$. Así mismo, hay una función que toma una serie diaria y la transforma a una serie mensual.

In [ ]:
def presion_vapor_sat(T):
    '''
        Parámetros
        ----------
        T : float
            Temperatura (C).

        Returns
        -------
        es : float
            Presión de vapor de saturación (Pa)
    '''
    if (T>=0):
        es = 611 * np.exp(17.27*T/(T + 237.3))
    else:
        es = 611 * np.exp(21.87*T/(T + 265.5))
    return es

def latente(T):
    '''
        Parámetros
        ----------
        T : float
            Temperatura (C)

        Returns
        -------
        lh : float
            Calor latente de evaporación/condensación (MJ kg^-1)
    '''
    return 2.501 - 0.00236 * T

def delta(T):
    '''
        Parámetros
        ----------
        T : float
            Temperatura (C)

        Returns
        -------
        D : float
            Gradiente de presión de vapor de saturación con la temperatura (kPa/C)
    '''
    D = 2508.3/((T + 237.3)**2) * np.exp(17.3*T/(T+237.3))
    return D

def psicrometrica(p, T, ca = 1e-3):
    '''
        Parámetros
        ----------
        T : float
            Temperatura (K)
        p : float
            presión atmosférica (kPa)
        ca : float
            Calor específico (MJ/kg K)

        Returns
        -------
        gamma : float
            Constante psicrométrica (kPa/C)
    '''
    lh = latente(T - 273.15)
    gamma = ca * p / (0.622 * lh)
    return gamma

def et0_fao(T, p, RH, Rn, u):
    '''
        Parámetros
        ----------
        T : float
            Temperatura (K)
        p : float
            presión atmosférica (kPa)
        RH : float
            Humedad relativa medida a 2 m (%)
        Rn : float
            Radiación neta (MJ/m2/d)
        u : float
            Velocidad del viento medida a 2 m (m/s)

        Returns
        -------
        et0 : float
            Evapotranspiración de referencia (mm/d)
    '''
    D = delta(T - 273.15)
    gamma = psicrometrica(p, T)
    es = presion_vapor_sat(T - 273.15)/1000 # Presión de vapor de saturación (kPa)
    et0 = (0.408 * D * Rn + gamma * 900/T * u * es * (1 - RH/100)) / (D + gamma * (1 + 0.34 * u))
    return et0

def daily2monthly(ts, method='sum'):
    '''
        Parámetros
        ----------
        ts : pandas DataFrame
            serie de valores diarios en un año [Fecha: {Mes día, año}, valor]
        method : string
            método de agregación de la serie: 'sum': suma
                                              'mean': promedio

        Returns
        -------
        ts_m : list
            lista con los valores mensuales desde enero a diciembre
    '''
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ts_m = []

    for m in months:
        temp = []

        for row in ts.iterrows():
            m_ = row[1][0].split(' ')[0]
            if m == m_:
                temp = temp + [row[1][1]]

        if method == 'sum':
            ret = sum(temp)
        elif method == 'mean':
            ret = np.mean(temp)

        ts_m = ts_m + [ret]
    
    return ts_m

Desarrolle en el siguiente bloque su código:

In [ ]:
# Serie mensual de factores efectivos de cultivo
kc = [1, 0.5, 0.4, 0.8, 1.1, 1.2, 0.55, 0.25, 0.15, 0.4, 0.6, 1.2]

# Lista con número de días en cada mes
n = [31, 28.5, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# ---- Convertir la serie diaria de temperatura a serie mensual --

# ----------------------------------------------------------------

# Importar las series meteorológicas mensuales
RH = np.loadtxt('humedad2050.csv')
Rn = np.loadtxt('radiacion2050.csv')
u = np.loadtxt('vel_viento2050.csv')

p = 57 # Presión atmosférica (kPa)

# Calcular ET0 y PET para cada mes

# ---- ingrese aquí su código -------------------------------------

# ----------------------------------------------------------------

**Parte 3 - Cálculo de la relación de mezcla cerca a la superficie (10 puntos)**

Un colega de su equipo planea utilizar un modelo físicamente basado de precipitación para evaluar los resultados del modelo climático regional y necesita para ello que usted le haga entrega de la serie mensual de la relación de mezcla cerca a la superficie en el año 2050 en su área de estudio. Grafique la serie de relación de mezcla obtenida en unidades de g/kg.

In [ ]:
# ---- ingrese aquí su código -----------------------------------

# ----------------------------------------------------------------

**Parte 4 - Cálculo de la recarga mensual de agua subterránea (40 puntos)**

Uno de sus compañeros está preparando un modelo hidrogeológico para su área de estudio y requiere valores mensuales promedio de recarga para el año 2050 con el fin de evaluar efectos de cambio climático sobre la disponibilidad de agua subterránea. 

Como una primera aproximación, su equipo de modelación cuenta con una relación empírica con la cual se pueden estimar valores mensuales multianuales de escorrentía en función de valores mensuales de precipitación y evapotranspiración potencial, dada por la siguiente expresión:

Si $P_m > PER_m$:

$$R_m = P_m[1-(PET_m/P_m)^{w}]$$

Si no:

$$R_m = 0$$

siendo $w=0.36$ en su área de estudio y $m$ el indicador del mes.

Escriba una función para calcular la escorrentía con la ecuación empírica presentada arriba y modifique el modelo de balance hídrico mensual tipo Thornthwaite para estimar los valores de recarga mensual de agua subterránea. La capacidad de almacenamiento máximo en el suelo $SM_{max}$ se estima en 113.6 mm.

**Nota:** Si obtiene valores negativos para la recarga, asigne un valor de 0 a dichos valores.

Finalmente, en una misma gráfica, presente las series de precipitación, PET, ET, humedad del suelo, escorrentía y recarga y describa brevemente los resultados.

En el siguiente bloque está el modelo de balance hídrico mensual.

In [ ]:
# --- Genere la función para estimar la escorrentía aquí --------------

# ---------------------------------------------------------------------

# ---- Convertir la serie diaria de precipitación a serie mensual -----

# ---------------------------------------------------------------------

# --- Calcule la escorrentía mensual aquí -----------------------------

# ---------------------------------------------------------------------

# --- Balance hídrico mensual (a modificar) ---------------------------
SMmax = 113.6
SM0 = 0
error = 1
tol = 1e-6

while error > tol:
    SM_ini = SM0
    ET = SM = Q = []
    for PET_i, P_i in zip(PET, P):
        if PET_i <= P_i:
            ET_i = PET_i
            SM_i = min((P_i - ET_i) + SM_ini, SMmax)
        else:
            dSM = SM_ini * (1 - np.exp(-(PET_i - P_i)/SMmax))
            ET_i = P_i + dSM
            SM_i = SM_ini - dSM
        
        SM_ini = SM_i
        ET = ET + [ET_i]
        SM = SM + [SM_i]
    
    error = abs(SM0 - SM_ini)
    SM0 = SM_ini   
# ------------------------------------------------------------------------

# -- Calcule aquí la recarga ---------------------------------------------

# ------------------------------------------------------------------------

In [ ]:
# -- Haga aquí la gráfica ------------------------------------------------

# ------------------------------------------------------------------------